In [1]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from simtk import unit, openmm
import argparse
import os
import time
import mdtraj as md
from tqdm import tqdm

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)



In [3]:
dir = "/data/chodera/zhangi/perses_benchmark/neq/15/16/"
phase = 'complex'
sim_number = 1
old_aa_name = 'asn'
new_aa_name = 'lys'
length = 1

# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics_core': x,
                             'lambda_electrostatics_core': x,
                             'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_bonds': x,
                             'lambda_angles': x,
                             'lambda_torsions': x}

# Define simulation parameters
# nsteps_eq = 25000 # 100 ps 
nsteps_eq = 1
nsteps_neq = int(length*250000) # 1 ns
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
cache_length = 1


In [4]:
# Read in vanilla htf
i = os.path.basename(os.path.dirname(dir))
with open(os.path.join(dir, f"{i}_{phase}.pickle"), 'rb') as f:
    htf = pickle.load(f)

# Read in lambda = 0 cache
with open(os.path.join(dir, f"{i}_{phase}_{old_aa_name}_{cache_length}ns_snapshots.npy"), 'rb') as f:
    subset_pos = np.load(f)
positions = subset_pos[sim_number - 1]
system = htf.hybrid_system

# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep)



In [5]:
# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)


In [15]:
context.getPlatform().getPropertyValue(context, 'CudaPrecision')

'mixed'